TO CREATE THE SPARK SESSION

In [1]:
from pyspark.sql import SparkSession
spark= SparkSession.builder.master("local[1]").appName("Assignment1.com").getOrCreate()


24/10/14 15:08:42 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/10/14 15:08:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 15:08:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


IMPORTING STATEMENT

In [20]:
from pyspark.sql.functions import col,monotonically_increasing_id,regexp_replace,split,size,sum,avg,regexp_extract,desc,collect_list,first
from pyspark.sql.types import *
# from pyspark.sql import functions as F

LOADING THE CSV FILE AS AN RDD  AND CONVERTING RDD TO DATAFRAME

In [3]:
data=spark.read.options(delimiter=",",header="true").csv("/home/ai/Documents/my files/assignment/hotels.csv")
rdd1=data.rdd
print("type:",type(rdd1))
df=rdd1.toDF()
df.show(1,truncate=False)

type: <class 'pyspark.rdd.RDD'>


+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
|Holding company|Headquarters  |Locations|Year established|Properties|Rooms     |Category|Managed brands|Franchised brands             |
+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
|Accor          |Paris (France)|Worldwide|1967            |5,300[4]  |778,000[4]|Luxury  |Raffles       |Banyan Tree Hótels and Resorts|
+---------------+--------------+---------+----------------+----------+----------+--------+--------------+------------------------------+
only showing top 1 row



TO CREATE A DATAFRAME WHERE IT CONTAINS  COLUMNS["Holding company", "Headquarters", "Locations", "Year established", "Properties", "Rooms"]

In [4]:
df1 = df.select("Holding company","Headquarters", "Locations", "Year established", "Properties", "Rooms")


In [5]:
for column in df1.columns:
    df1 = df1.withColumn(column, regexp_replace(column, "\\[|\\]", ""))
df1.show(1)
# df1.filter(col("Holding company")=="Red Roof Inn ").show()

+---------------+--------------+---------+----------------+----------+--------+
|Holding company|  Headquarters|Locations|Year established|Properties|   Rooms|
+---------------+--------------+---------+----------------+----------+--------+
|          Accor|Paris (France)|Worldwide|            1967|    5,3004|778,0004|
+---------------+--------------+---------+----------------+----------+--------+
only showing top 1 row



only 1 row per "Holding company", be sure to concatenate info if they appear in more than one row/cell)

In [6]:
df1.show(1)

+---------------+--------------+---------+----------------+----------+--------+
|Holding company|  Headquarters|Locations|Year established|Properties|   Rooms|
+---------------+--------------+---------+----------------+----------+--------+
|          Accor|Paris (France)|Worldwide|            1967|    5,3004|778,0004|
+---------------+--------------+---------+----------------+----------+--------+
only showing top 1 row



NO MISSING IN ANY COLUMNS
imputing the null values with mode


CLEAN INFORMATION ABOUT THE REFERENCES (THEY APPEAR AS[NUMBER])


In [7]:
df1.printSchema()

root
 |-- Holding company: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- Year established: string (nullable = true)
 |-- Properties: string (nullable = true)
 |-- Rooms: string (nullable = true)



CREATE A NEW COLUMN ASSOCIATED "HEADQUARTERS COUNTRY"(NOW IS PART OF THE COLUMN "HEADQUARTERS")

In [8]:
# from pyspark.sql import functions as F

df1 = df1.withColumn(
    "Headquarters country",
    regexp_extract(col("Headquarters"), r'\(([^)]+)\)', 1)  # Extract text within parentheses
)
# # to remove the country from the headquarters
# df1 = df1.withColumn(
#     "Headquarters",
#     F.regexp_replace(col("Headquarters"), r'\s*\(.*\)', '')  # Remove the text in parentheses
# )


df1.show(1,truncate=False)
# Show the result with the new column
df1.select("Headquarters", "Headquarters country").filter(col("Headquarters")!="null").show(10,truncate=False)

+---------------+--------------+---------+----------------+----------+--------+--------------------+
|Holding company|Headquarters  |Locations|Year established|Properties|Rooms   |Headquarters country|
+---------------+--------------+---------+----------------+----------+--------+--------------------+
|Accor          |Paris (France)|Worldwide|1967            |5,3004    |778,0004|France              |
+---------------+--------------+---------+----------------+----------+--------+--------------------+
only showing top 1 row

+--------------------+--------------------+
|Headquarters        |Headquarters country|
+--------------------+--------------------+
|Paris (France)      |France              |
|Baar, Switzerland   |                    |
|Tokyo (Japan)       |Japan               |
|Jakarta, (Indonesia)|Indonesia           |
|Singapore           |                    |
|Karachi (Pakistan)  |Pakistan            |
|Singapore           |                    |
|Paris (France)      |France   

SECOND DATAFRAME

THE SECOND DATAFRAME WITH COLUMNSA["HOLDING COMPANY","CATEGORY","MANAGED BRANDS","FRANCHISED BRANDS"]

In [9]:
df2 = df.select(
    "Holding company",
    "Category",           # This column should already exist in df
    "Managed brands",     # This column should already exist in df
    "Franchised brands"   # This column should already exist in df
)
df2.show(1, truncate=False)


+---------------+--------+--------------+------------------------------+
|Holding company|Category|Managed brands|Franchised brands             |
+---------------+--------+--------------+------------------------------+
|Accor          |Luxury  |Raffles       |Banyan Tree Hótels and Resorts|
+---------------+--------+--------------+------------------------------+
only showing top 1 row



ONLY THE FRANCHISED BRANDS COLUMN CAN POSSESS NULL VALUES

In [10]:
#droping out the rows with null values
filtered_df = df2.filter(col("Holding company").isNotNull())

# Show the filtered DataFrame
filtered_df.show(5,truncate=False)

#imputing the category column with mode value
mode_value = filtered_df.groupBy("Category").count().orderBy(desc("count")).first()[0]

# Step 2: Fill nulls in the "Category" column with the mode value
imputed_df = filtered_df.fillna({"Category": mode_value})

# Show the final DataFrame
imputed_df.show(5,truncate=False)

+-------------------------+--------+------------------------------------------------------------------------------------------------------------------------+------------------------------+
|Holding company          |Category|Managed brands                                                                                                          |Franchised brands             |
+-------------------------+--------+------------------------------------------------------------------------------------------------------------------------+------------------------------+
|Accor                    |Luxury  |Raffles                                                                                                                 |Banyan Tree Hótels and Resorts|
|Aman Resorts             |Luxury  |Aman                                                                                                                    |NULL                          |
|APA Group                |Midscale|Coast Hotels       

SPECIFYING THE BEST TYPE AND NULLABLE FOR EACH NCOLUMN

In [11]:
# Define the schema with data types and nullability
schema = StructType([
    StructField("Holding company", StringType(), nullable=True),
    StructField("Category", StringType(), nullable=True),  # Assuming it's nullable
    StructField("Managed brands", StringType(), nullable=True),  # Assuming it's nullable
    StructField("Franchised brands", StringType(), nullable=True)  # Assuming it's nullable
])

# Create or modify the DataFrame with the defined schema

df_with_schema = spark.createDataFrame(imputed_df.rdd, schema)

# Show the schema
df_with_schema.printSchema()

root
 |-- Holding company: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Managed brands: string (nullable = true)
 |-- Franchised brands: string (nullable = true)



USE PYSPARK DATAFRAME GROUPBY TO COMPUTE

  TOTAL NUMBER OF ROOMS PER "HEADQUARTERD COUNTRY"

  AVERAGE NUMBER OF PROPERTIES PER "HEADQUARTERED COUNTRY"

  NUMBER OF "MANAGED BRANDS" PER "HOLDING COMPANY" AND "CATEGORY"

In [12]:
print(df1.columns)
df1.printSchema()

['Holding company', 'Headquarters', 'Locations', 'Year established', 'Properties', 'Rooms', 'Headquarters country']
root
 |-- Holding company: string (nullable = true)
 |-- Headquarters: string (nullable = true)
 |-- Locations: string (nullable = true)
 |-- Year established: string (nullable = true)
 |-- Properties: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Headquarters country: string (nullable = true)



In [13]:
df1 = df1.withColumn('Properties', regexp_replace(col('Properties'), ',', ''))
df1 = df1.withColumn('Rooms', regexp_replace(col('Rooms'), ',', ''))

# Cast columns to integer data types
df1 = df1.withColumn("Properties", col("Properties").cast("integer"))
df1 = df1.withColumn("Rooms", col("Rooms").cast("integer"))

In [14]:
rooms_per_country = df1.groupBy("Headquarters country").agg(
    sum("Rooms").alias("Total Rooms")
)

# Average number of properties per "Headquarters country"
avg_properties_per_country = df1.groupBy("Headquarters country").agg(
    avg("Properties").alias("Average Properties")
)

# Show results
rooms_per_country.show(10,truncate=False)
avg_properties_per_country.show(10,truncate=False)

+--------------------+-----------+
|Headquarters country|Total Rooms|
+--------------------+-----------+
|Sweden              |49983107   |
|Germany             |2215116    |
|France              |8010015    |
|Sri Lanka           |255627     |
|NULL                |NULL       |
|Belgium             |NULL       |
|China               |139899449  |
|India               |50105310   |
|Norway              |NULL       |
|Spain               |63228856   |
+--------------------+-----------+
only showing top 10 rows

+--------------------+------------------+
|Headquarters country|Average Properties|
+--------------------+------------------+
|Sweden              |67405.25          |
|Germany             |7663.0            |
|France              |27407.5           |
|Sri Lanka           |22.0              |
|NULL                |NULL              |
|Belgium             |NULL              |
|China               |351762.25         |
|India               |75160.66666666667 |
|Norway              |

In [15]:
#The first 5 rows of the second DataFrame
# df2.show(5)
#Adding row_number for rows
second_df = df2.withColumn("row_number",monotonically_increasing_id())
second_df.show(1)

+---------------+--------+--------------+--------------------+----------+
|Holding company|Category|Managed brands|   Franchised brands|row_number|
+---------------+--------+--------------+--------------------+----------+
|          Accor|  Luxury|       Raffles|Banyan Tree Hótel...|         0|
+---------------+--------+--------------+--------------------+----------+
only showing top 1 row



In [16]:
# Filter rows where the "Holding company" column contains string values
rows_with_string_values_df2 = second_df.filter(col("Holding company").isNotNull() & col("Holding company").cast("string").rlike("[a-zA-Z]"))

# Select row numbers
row_numbers_with_string_values_df2 = rows_with_string_values_df2.select("row_number").rdd.flatMap(lambda x: x).collect()

# Display the row numbers
print("Row numbers with string values in the Holding company column:", row_numbers_with_string_values_df2)

Row numbers with string values in the Holding company column: [0, 36, 37, 40, 41, 56, 57, 61, 65, 66, 79, 84, 91, 92, 101, 114, 115, 119, 121, 125, 130, 138, 139, 141, 142, 146, 147, 151, 152, 157, 170, 173, 177, 185, 191, 205, 207, 208, 209, 222, 223, 225, 230, 235, 236, 237, 246, 250, 251, 280, 286, 292, 294, 301, 305, 311, 313, 317, 320, 332, 342, 347, 351, 359, 369, 373, 379, 381, 383, 387, 393, 401, 405, 406, 407, 410, 414, 418, 422, 423, 424, 425, 426, 428, 447]


In [17]:
from functools import reduce
from pyspark.sql import DataFrame

In [21]:
####Divding and combining the Dataframe based on row_numbers_with_string_values_df2####
separted_df2=[]
for i in range(0,len(row_numbers_with_string_values_df2)-1):
    start=row_numbers_with_string_values_df2[i]
    final=row_numbers_with_string_values_df2[i+1]
    # print(start,final)
    # Select rows initial to final and concatenate values into a single row
    concatenated_df = second_df.filter((col("row_number") >= start) & (col("row_number") < final)) \
                    .select([collect_list(col).alias(col) for col in second_df.columns]) \
                    .agg(*[first(col).alias(col) for col in second_df.columns])
    
    # concatenated_df.show(1)
    separted_df2.append(concatenated_df)


# print(separted_df2)
# Combine DataFrames into a single DataFrame
combined_df2 = reduce(DataFrame.union, separted_df2)
# combined_df2.show()

# droping the row_number
combined_df2_without_column = combined_df2.drop("row_number")



In [22]:
df2_columns=combined_df2_without_column.columns
for column in df2_columns:
    # Change the data type of the column to string
    combined_df2_without_column = combined_df2_without_column.withColumn(column, col(column).cast("string"))

    # Remove square brackets from the string
    combined_df2_without_column = combined_df2_without_column.withColumn(column, regexp_replace(column, "\\[|\\]", ""))
combined_df2_without_column.show(truncate=False)

24/10/14 15:10:12 WARN DAGScheduler: Broadcasting large task binary with size 1280.1 KiB
24/10/14 15:10:12 WARN DAGScheduler: Broadcasting large task binary with size 1280.1 KiB
24/10/14 15:10:12 WARN DAGScheduler: Broadcasting large task binary with size 1280.1 KiB


+---------------------------+------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------+
|Holding company            |Category                                                                                  |Managed brands                                                                                                                                                                                                                                                                                                                                          

In [23]:
# Dropping the rows with null values and empty string "" 
combined_df2_without_column = combined_df2_without_column.dropna(subset=["Category"])

combined_df2_without_column = combined_df2_without_column.filter(col("Category") != "")

combined_df2_without_column = combined_df2_without_column.dropna(subset=["Holding company"])

combined_df2_without_column = combined_df2_without_column.filter(col("Holding company") != "")

# Dropping the rows with null values and empty string "" 

combined_df2_without_column = combined_df2_without_column.dropna(subset=["Managed brands"])

combined_df2_without_column = combined_df2_without_column.filter(col("Managed brands") != "")



In [210]:

# Split the "Rooms" column by commas and count the number of elements
No_Managedbrands = combined_df2_without_column.withColumn("Managed Brands", size(split(col("Managed Brands"), ","))) \
    .groupBy("Holding Company","Category") \
    .agg(sum("Managed Brands").alias("Total_Managed Brands"))

# Show the result
No_Managedbrands.show(truncate=False)

24/09/30 12:25:32 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB


+---------------------------+------------------------------------------------------------------------------------------+--------------------+
|Holding Company            |Category                                                                                  |Total_Managed Brands|
+---------------------------+------------------------------------------------------------------------------------------+--------------------+
|Accor                      |Luxury, Premium, Midscale, Economy                                                        |36                  |
|Aman Resorts               |Luxury                                                                                    |1                   |
|APA Group                  |Midscale                                                                                  |2                   |
|Ascott Limited (The)       |Luxury, Premium, Upscale, Upper-Midscale, Midscale, Lower-Midscale, Economy, Uncategorized|15                  |
|Avari

24/09/30 12:25:36 WARN DAGScheduler: Broadcasting large task binary with size 1961.1 KiB
